# Variational RX Gate Optimization to Approximate Pauli-X

This notebook demonstrates how to optimize a parameterized single-qubit RX gate to match the Pauli-X gate using a cost function based on **process fidelity**.

## Import Libraries

We will use Qiskit for quantum circuits and simulation, and NumPy for numerical computations.

In [48]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit-aer

In [49]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.quantum_info import process_fidelity, Operator
import numpy as np

## Define the Simulator

We will use the statevector simulator from Qiskit AER to get the exact quantum state after applying the RX gate.

In [50]:
# Initialize the Aer statevector simulator
sim = AerSimulator(method='statevector')

## Define the RX(θ) Gate Circuit

The RX gate rotates a qubit around the X-axis by angle θ. Mathematically, it is represented as:

$$
RX(\theta) = e^{-i\frac{\theta}{2}X} =
\begin{pmatrix}
\cos\frac{\theta}{2} & -isin\frac{\theta}{2} \\
-i\sin\frac{\theta}{2} & \cos\frac{\theta}{2}
\end{pmatrix}
$$

In [51]:
def RX_circuit(theta):
    qc = QuantumCircuit(1)
    qc.rx(theta, 0)
    # qc.save_statevector()
    return qc

## Define the Cost Function (Process Fidelity)

To compare RX(θ) with X, we use the **process fidelity**:

$ F(\theta) = process\_fidelity(RX(\theta), X) $

The cost function is then:

$ C(\theta) = 1 - F(\theta) $

Minimizing this cost drives RX(θ) to approximate the Pauli-X gate.

In [52]:
def cost(theta):
    # Build RX circuit and get its unitary as an Operator object
    RX_op = Operator(RX_circuit(theta))
    # Target Pauli-X gate as an Operator object
    X_op = Operator(np.array([[0,1],[1,0]], dtype=complex))
    # Compute process fidelity, passing Operator objects
    F = process_fidelity(RX_op, X_op)
    return 1 - F

## Parameter-Shift Gradient

The derivative of the cost function can be estimated using the **parameter-shift rule**:

$ \frac{\partial C}{\partial \theta} \approx \frac{C(\theta + \pi/2) - C(\theta - \pi/2)}{2} $

In [53]:
def gradient(theta):
    shift = np.pi / 2
    return 0.5 * (cost(theta + shift) - cost(theta - shift))

## Classical Optimization Loop

We perform gradient descent to minimize the cost function and find θ that makes RX(θ) ≈ X.

In [69]:
theta = np.random.uniform(0, 2*np.pi)
learning_rate = 0.2
cost_history = []

for it in range(100):
    grad = gradient(theta)
    theta -= learning_rate * grad
    current_cost = cost(theta)
    cost_history.append(current_cost)
    if it % 5 == 0:
        print(f"Iter {it:02d} | theta = {theta:.6f} | cost = {cost(theta):.8f}")

print("\nFinal θ:", theta)
print("Expected θ ≈ π:", np.pi)
print("Final cost:", cost(theta))

Iter 00 | theta = 2.055017 | cost = 0.26724062
Iter 05 | theta = 2.449823 | cost = 0.11494088
Iter 10 | theta = 2.720480 | cost = 0.04368257
Iter 15 | theta = 2.890112 | cost = 0.01572744
Iter 20 | theta = 2.992498 | cost = 0.00554698
Iter 25 | theta = 3.053430 | cost = 0.00194192
Iter 30 | theta = 3.089508 | cost = 0.00067806
Iter 35 | theta = 3.110832 | cost = 0.00023654
Iter 40 | theta = 3.123428 | cost = 0.00008249
Iter 45 | theta = 3.130866 | cost = 0.00002876
Iter 50 | theta = 3.135259 | cost = 0.00001003
Iter 55 | theta = 3.137852 | cost = 0.00000350
Iter 60 | theta = 3.139384 | cost = 0.00000122
Iter 65 | theta = 3.140289 | cost = 0.00000043
Iter 70 | theta = 3.140823 | cost = 0.00000015
Iter 75 | theta = 3.141138 | cost = 0.00000005
Iter 80 | theta = 3.141324 | cost = 0.00000002
Iter 85 | theta = 3.141434 | cost = 0.00000001
Iter 90 | theta = 3.141499 | cost = 0.00000000
Iter 95 | theta = 3.141537 | cost = 0.00000000

Final θ: 3.141556382462133
Expected θ ≈ π: 3.14159265358979

## Summary

- We defined a variational RX(θ) gate and a cost function based on process fidelity.
- We used the parameter-shift rule to compute gradients.
- Gradient descent successfully finds θ ≈ π, so RX(θ) approximates the Pauli-X gate.
- This notebook demonstrates a simple **variational quantum gate synthesis** method.